In [1]:
import pandas as pd
import numpy as np
from sklearn import metrics
import tensorflow as tf
import binascii
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation

In [2]:
# import json
# import csv

# with open('./data/json-data/pcap-wordpress1.json') as json_file:
#     jsondata = json.load(json_file)

In [3]:
# data_file = open('./data/pcap.csv', 'w', newline='')
# csv_writer = csv.writer(data_file)

# list_of_headers = []
 
# csv_writer.writerow(jsondata[0].keys())

# for data in jsondata:
#     csv_writer.writerow(data.values())
    
# data_file.close()

In [4]:
#read the csvs

#this is the wordpress one
csv_data = pd.read_csv('./data/pcap.csv')

#this is the one with nmap packets from a test a while ago
csv_data_nmap = pd.read_csv('./data/pcap-nmap.csv')

#using just using the rows with nmap packets (the only packets with my device IP address help me identify it)
csv_data_nmap = csv_data_nmap.loc[csv_data_nmap['ip.dst'] == '89.100.107.148']

#creating new attack column on both dataframes
csv_data['attack'] = [0]*csv_data.length
csv_data_nmap['attack'] = [1]*csv_data_nmap.length

#combining the two dataframes
csv_data = csv_data.append([csv_data_nmap])

csv_data.reset_index(drop=True, inplace=True)

#removing the fields where protocal is NULL
csv_data = csv_data[csv_data['protocol'].notna()] 

#also removing ethernet type as its all 2048 
csv_data = csv_data.drop(['eth.type'], axis = 1)

/var/folders/x4/thfll6l51jv22hkn1948hhgh0000gn/T/ipykernel_79452/1294898060.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  csv_data = csv_data.append([csv_data_nmap])


In [5]:
csv_data

,eth.dst,eth.src,ip.dst,ip.flags,ip.ihl,ip.len,ip.proto,ip.src,ip.tos,ip.ttl,ip.version,length,load,load.count,protocol,protocol.dport,protocol.sport,attack
0,a6:4d:66:07:0e:f8,16:78:bc:af:56:36,10.244.0.11,DF,5.0,82.0,6.0,10.240.0.4,0.0,64.0,4.0,96,17 03 03 00 19 f5 9c c1,30.0,TCP,49190.0,10250.0,0
1,a6:4d:66:07:0e:f8,16:78:bc:af:56:36,10.244.0.11,DF,5.0,158.0,6.0,10.240.0.4,0.0,64.0,4.0,172,17 03 03 00 65 7a 07 8b,106.0,TCP,49190.0,10250.0,0
2,16:78:bc:af:56:36,a6:4d:66:07:0e:f8,52.143.28.136,DF,5.0,216.0,6.0,10.244.0.11,16.0,64.0,4.0,230,b9 4f 72 cb 17 e2 e0 a6,164.0,TCP,9000.0,54822.0,0
3,a6:4d:66:07:0e:f8,16:78:bc:af:56:36,10.244.0.11,DF,5.0,52.0,6.0,52.143.28.136,0.0,62.0,4.0,66,NaN,NaN,TCP,54822.0,9000.0,0
4,a6:4d:66:07:0e:f8,16:78:bc:af:56:36,10.244.0.11,DF,5.0,184.0,6.0,52.143.28.136,0.0,62.0,4.0,198,25 e8 57 a5 05 11 dc b7,132.0,TCP,54822.0,9000.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43127,42:1b:a1:91:67:1c,1a:37:6a:88:60:f6,10.244.0.2,DF,5.0,52.0,6.0,10.244.0.6,0.0,64.0,4.0,66,NaN,NaN,TCP,47022.0,25227.0,0
43128,ca:10:2c:11:5d:b1,62:a1:17:9b:b1:4d,89.100.107.148,DF,5.0,44.0,6.0,10.244.0.3,0.0,64.0,4.0,58,NaN,NaN,TCP,51626.0,5000.0,58
43129,ca:10:2c:11:5d:b1,62:a1:17:9b:b1:4d,89.100.107.148,DF,5.0,44.0,6.0,10.244.0.3,0.0,64.0,4.0,58,NaN,NaN,TCP,51036.0,5000.0,58
43130,ca:10:2c:11:5d:b1,62:a1:17:9b:b1:4d,89.100.107.148,DF,5.0,44.0,6.0,10.244.0.3,0.0,64.0,4.0,58,NaN,NaN,TCP,57491.0,5000.0,58


In [6]:
csv_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 42864 entries, 0 to 43131
Data columns (total 18 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   eth.dst         42864 non-null  object 
 1   eth.src         42864 non-null  object 
 2   ip.dst          42864 non-null  object 
 3   ip.flags        42618 non-null  object 
 4   ip.ihl          42864 non-null  float64
 5   ip.len          42864 non-null  float64
 6   ip.proto        42864 non-null  float64
 7   ip.src          42864 non-null  object 
 8   ip.tos          42864 non-null  float64
 9   ip.ttl          42864 non-null  float64
 10  ip.version      42864 non-null  float64
 11  length          42864 non-null  int64  
 12  load            18641 non-null  object 
 13  load.count      18641 non-null  float64
 14  protocol        42864 non-null  object 
 15  protocol.dport  42864 non-null  float64
 16  protocol.sport  42864 non-null  float64
 17  attack          42864 non-null 

In [7]:
csv_data.columns

Index(['eth.dst', 'eth.src', 'ip.dst', 'ip.flags', 'ip.ihl', 'ip.len',
       'ip.proto', 'ip.src', 'ip.tos', 'ip.ttl', 'ip.version', 'length',
       'load', 'load.count', 'protocol', 'protocol.dport', 'protocol.sport',
       'attack'],
      dtype='object')

In [8]:
#trying lambda function
list_of_octects = '0a:dc:5b:dc:e2:46'.split(':')

In [9]:
hex_lambda = lambda x: int(x,16)

In [10]:
list(map(hex_lambda, list_of_octects))

[10, 220, 91, 220, 226, 70]

In [11]:
set(csv_data['ip.flags'])

{'DF', nan}

In [12]:
csv_data['load'].fillna(0.0, inplace = True)
csv_data['load.count'].fillna(0.0, inplace = True)

# csv_data['ip.flags'].fillna("0", inplace = True)

In [13]:
csv_data

,eth.dst,eth.src,ip.dst,ip.flags,ip.ihl,ip.len,ip.proto,ip.src,ip.tos,ip.ttl,ip.version,length,load,load.count,protocol,protocol.dport,protocol.sport,attack
0,a6:4d:66:07:0e:f8,16:78:bc:af:56:36,10.244.0.11,DF,5.0,82.0,6.0,10.240.0.4,0.0,64.0,4.0,96,17 03 03 00 19 f5 9c c1,30.0,TCP,49190.0,10250.0,0
1,a6:4d:66:07:0e:f8,16:78:bc:af:56:36,10.244.0.11,DF,5.0,158.0,6.0,10.240.0.4,0.0,64.0,4.0,172,17 03 03 00 65 7a 07 8b,106.0,TCP,49190.0,10250.0,0
2,16:78:bc:af:56:36,a6:4d:66:07:0e:f8,52.143.28.136,DF,5.0,216.0,6.0,10.244.0.11,16.0,64.0,4.0,230,b9 4f 72 cb 17 e2 e0 a6,164.0,TCP,9000.0,54822.0,0
3,a6:4d:66:07:0e:f8,16:78:bc:af:56:36,10.244.0.11,DF,5.0,52.0,6.0,52.143.28.136,0.0,62.0,4.0,66,0.0,0.0,TCP,54822.0,9000.0,0
4,a6:4d:66:07:0e:f8,16:78:bc:af:56:36,10.244.0.11,DF,5.0,184.0,6.0,52.143.28.136,0.0,62.0,4.0,198,25 e8 57 a5 05 11 dc b7,132.0,TCP,54822.0,9000.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43127,42:1b:a1:91:67:1c,1a:37:6a:88:60:f6,10.244.0.2,DF,5.0,52.0,6.0,10.244.0.6,0.0,64.0,4.0,66,0.0,0.0,TCP,47022.0,25227.0,0
43128,ca:10:2c:11:5d:b1,62:a1:17:9b:b1:4d,89.100.107.148,DF,5.0,44.0,6.0,10.244.0.3,0.0,64.0,4.0,58,0.0,0.0,TCP,51626.0,5000.0,58
43129,ca:10:2c:11:5d:b1,62:a1:17:9b:b1:4d,89.100.107.148,DF,5.0,44.0,6.0,10.244.0.3,0.0,64.0,4.0,58,0.0,0.0,TCP,51036.0,5000.0,58
43130,ca:10:2c:11:5d:b1,62:a1:17:9b:b1:4d,89.100.107.148,DF,5.0,44.0,6.0,10.244.0.3,0.0,64.0,4.0,58,0.0,0.0,TCP,57491.0,5000.0,58


In [14]:
#Plan 
# For ethernet: make different columns for octects, convert hex to int and normalize
hex_lambda = lambda x: int(x,16)

##eth_dst
eth_dst_list = list(csv_data['eth.dst'])

octet_1_dst = []
octet_2_dst = []
octet_3_dst = []
octet_4_dst = []
octet_5_dst = []
octet_6_dst = []

for eth in eth_dst_list:
    int_eth_list = list(map(hex_lambda, eth.split(':')))
    octet_1_dst.append(int_eth_list[0])
    octet_2_dst.append(int_eth_list[1])
    octet_3_dst.append(int_eth_list[2])
    octet_4_dst.append(int_eth_list[3])
    octet_5_dst.append(int_eth_list[4])
    octet_6_dst.append(int_eth_list[5])

# creating a final dataframe that will be used to store modified data
final_df = pd.DataFrame(octet_1_dst, columns =['octet_1_eth_dst'])
final_df['octet_2_eth_dst'] = octet_2_dst
final_df['octet_3_eth_dst'] = octet_3_dst
final_df['octet_4_eth_dst'] = octet_4_dst
final_df['octet_5_eth_dst'] = octet_5_dst
final_df['octet_6_eth_dst'] = octet_6_dst

In [15]:
##eth_src
eth_src_list = list(csv_data['eth.src'])

octet_1_src = []
octet_2_src = []
octet_3_src = []
octet_4_src = []
octet_5_src = []
octet_6_src = []

for eth in eth_src_list:
    int_eth_list = list(map(hex_lambda, eth.split(':')))
    octet_1_src.append(int_eth_list[0])
    octet_2_src.append(int_eth_list[1])
    octet_3_src.append(int_eth_list[2])
    octet_4_src.append(int_eth_list[3])
    octet_5_src.append(int_eth_list[4])
    octet_6_src.append(int_eth_list[5])
    
final_df['octet_1_eth_src'] = octet_1_src 
final_df['octet_2_eth_src'] = octet_2_src
final_df['octet_3_eth_src'] = octet_3_src
final_df['octet_4_eth_src'] = octet_4_src
final_df['octet_5_eth_src'] = octet_5_src
final_df['octet_6_eth_src'] = octet_6_src


In [16]:
## ip.dst and ip.src 

ip_src_list = list(map(str,list(csv_data['ip.src'])))

octet_1_src = []
octet_2_src = []
octet_3_src = []
octet_4_src = []

for eth in ip_src_list:
    int_ip_list = list(eth.split('.'))
    octet_1_src.append(int_ip_list[0])
    octet_2_src.append(int_ip_list[1])
    octet_3_src.append(int_ip_list[2])
    octet_4_src.append(int_ip_list[3])

final_df['octet_1_ip_src'] = octet_1_src 
final_df['octet_2_ip_src'] = octet_2_src
final_df['octet_3_ip_src'] = octet_3_src
final_df['octet_4_ip_src'] = octet_4_src



In [17]:
## ip.dst and ip.src

ip_dst_list = list(csv_data['ip.dst'])

octet_1_dst = []
octet_2_dst = []
octet_3_dst = []
octet_4_dst = []

for eth in ip_dst_list:
    int_ip_list = list(eth.split('.'))
    octet_1_dst.append(int_ip_list[0])
    octet_2_dst.append(int_ip_list[1])
    octet_3_dst.append(int_ip_list[2])
    octet_4_dst.append(int_ip_list[3])

final_df['octet_1_ip_dst'] = octet_1_dst
final_df['octet_2_ip_dst'] = octet_2_dst
final_df['octet_3_ip_dst'] = octet_3_dst
final_df['octet_4_ip_dst'] = octet_4_dst

In [18]:
#  4   ip.flags        42860 non-null  object    DROP
#  5   ip.ihl          42860 non-null  float64   DROP
#  6   ip.len          42860 non-null  float64   DONE 
#  7   ip.proto        42860 non-null  float64   DROP
#  9   ip.tos          42860 non-null  float64   DONE
#  10  ip.ttl          42860 non-null  float64   DONE
#  11  ip.version   DROP

# drop ip version as all are 4
# drop ihl as everything is 5
# drop flags as well, might not we useful lets see

In [19]:
#fields being included in final_csv 
# not including load at the moment, will include in further iterations
final_df['ip.len'] = list(csv_data['ip.len'])
final_df['ip.tos'] = list(csv_data['ip.tos'])
final_df['ip.ttl'] = list(csv_data['ip.ttl'])
final_df['length'] = list(csv_data['length'])
final_df['load.count'] = list(csv_data['load.count'])
final_df['attack'] = list(csv_data['attack'])

In [20]:
#one hot encoding protocol
dummy_protocol = pd.get_dummies(csv_data['protocol'])

In [21]:
final_df['TCP'] = list(dummy_protocol['TCP'])
final_df['UDP'] = list(dummy_protocol['UDP'])
final_df['protocol.dport'] = list(csv_data['protocol.dport'])
final_df['protocol.sport'] = list(csv_data['protocol.sport'])

In [22]:
final_df = final_df.astype('float64')

In [23]:
final_df.columns

Index(['octet_1_eth_dst', 'octet_2_eth_dst', 'octet_3_eth_dst',
       'octet_4_eth_dst', 'octet_5_eth_dst', 'octet_6_eth_dst',
       'octet_1_eth_src', 'octet_2_eth_src', 'octet_3_eth_src',
       'octet_4_eth_src', 'octet_5_eth_src', 'octet_6_eth_src',
       'octet_1_ip_src', 'octet_2_ip_src', 'octet_3_ip_src', 'octet_4_ip_src',
       'octet_1_ip_dst', 'octet_2_ip_dst', 'octet_3_ip_dst', 'octet_4_ip_dst',
       'ip.len', 'ip.tos', 'ip.ttl', 'length', 'load.count', 'attack', 'TCP',
       'UDP', 'protocol.dport', 'protocol.sport'],
      dtype='object')

In [24]:
#standardizing the data
from sklearn.preprocessing import StandardScaler

#columns to preprocess (exclude categorial columns like tcp, udp and attack)
num_cols = ['octet_1_eth_dst', 'octet_2_eth_dst', 'octet_3_eth_dst',
       'octet_4_eth_dst', 'octet_5_eth_dst', 'octet_6_eth_dst',
       'octet_1_eth_src', 'octet_2_eth_src', 'octet_3_eth_src',
       'octet_4_eth_src', 'octet_5_eth_src', 'octet_6_eth_src',
       'octet_1_ip_dst', 'octet_2_ip_dst', 'octet_3_ip_dst', 'octet_4_ip_dst',
       'ip.len', 'ip.tos', 'ip.ttl', 'length', 'load.count',
       'protocol.dport', 'protocol.sport']

for i in num_cols:
    scale = StandardScaler().fit(final_df[[i]])
    final_df[i] = scale.transform(final_df[[i]])


In [25]:
final_df

,octet_1_eth_dst,octet_2_eth_dst,octet_3_eth_dst,octet_4_eth_dst,octet_5_eth_dst,octet_6_eth_dst,octet_1_eth_src,octet_2_eth_src,octet_3_eth_src,octet_4_eth_src,...,ip.len,ip.tos,ip.ttl,length,load.count,attack,TCP,UDP,protocol.dport,protocol.sport
0,3.177013,-1.369874,0.333637,-3.358735,-1.860428,1.660901,-0.249163,-0.376705,1.298027,-0.219630,...,-0.220804,-0.700187,-0.053115,-0.220804,-0.217723,0.0,1.0,0.0,1.110607,-0.733359
1,3.177013,-1.369874,0.333637,-3.358735,-1.860428,1.660901,-0.249163,-0.376705,1.298027,-0.219630,...,-0.195153,-0.700187,-0.053115,-0.195153,-0.192087,0.0,1.0,0.0,1.110607,-0.733359
2,-0.171393,-0.618852,1.945527,-0.393830,-0.827084,-1.038936,2.970973,-1.244290,0.005170,-3.059465,...,-0.175576,3.017263,-0.053115,-0.175576,-0.172523,0.0,1.0,0.0,-0.729302,1.284677
3,3.177013,-1.369874,0.333637,-3.358735,-1.860428,1.660901,-0.249163,-0.376705,1.298027,-0.219630,...,-0.230930,-0.700187,-0.286207,-0.230930,-0.227843,0.0,1.0,0.0,1.368442,-0.789954
4,3.177013,-1.369874,0.333637,-3.358735,-1.860428,1.660901,-0.249163,-0.376705,1.298027,-0.219630,...,-0.186377,-0.700187,-0.286207,-0.186377,-0.183317,0.0,1.0,0.0,1.368442,-0.789954
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42859,0.851731,-2.243155,1.439469,-0.923277,-0.583100,-1.400770,-0.159715,-1.688171,0.065303,-0.878878,...,-0.230930,-0.700187,-0.053115,-0.230930,-0.227843,0.0,1.0,0.0,1.011355,-0.055263
42860,4.014114,-2.435277,-0.753451,-3.182253,-0.726620,0.672817,1.450353,0.450527,-1.182455,-0.557706,...,-0.233630,-0.700187,-0.053115,-0.233630,-0.227843,58.0,1.0,0.0,1.222128,-0.971057
42861,4.014114,-2.435277,-0.753451,-3.182253,-0.726620,0.672817,1.450353,0.450527,-1.182455,-0.557706,...,-0.233630,-0.700187,-0.053115,-0.233630,-0.227843,58.0,1.0,0.0,1.195117,-0.971057
42862,4.014114,-2.435277,-0.753451,-3.182253,-0.726620,0.672817,1.450353,0.450527,-1.182455,-0.557706,...,-0.233630,-0.700187,-0.053115,-0.233630,-0.227843,58.0,1.0,0.0,1.490629,-0.971057


In [26]:
#for some reason the attack has value of 58.0? wierd but can still create a mask 
normal_mask = final_df['attack']!=58.0
attack_mask = final_df['attack']==58.0

df_normal = final_df[normal_mask]
df_attack = final_df[attack_mask]

In [27]:
df_normal.drop('attack',axis=1,inplace=True)

/var/folders/x4/thfll6l51jv22hkn1948hhgh0000gn/T/ipykernel_79452/4061767891.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_normal.drop('attack',axis=1,inplace=True)


In [28]:
df_attack.drop('attack',axis=1,inplace=True)

/var/folders/x4/thfll6l51jv22hkn1948hhgh0000gn/T/ipykernel_79452/4004614948.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_attack.drop('attack',axis=1,inplace=True)


In [29]:
df_normal

,octet_1_eth_dst,octet_2_eth_dst,octet_3_eth_dst,octet_4_eth_dst,octet_5_eth_dst,octet_6_eth_dst,octet_1_eth_src,octet_2_eth_src,octet_3_eth_src,octet_4_eth_src,...,octet_4_ip_dst,ip.len,ip.tos,ip.ttl,length,load.count,TCP,UDP,protocol.dport,protocol.sport
0,3.177013,-1.369874,0.333637,-3.358735,-1.860428,1.660901,-0.249163,-0.376705,1.298027,-0.219630,...,-0.309108,-0.220804,-0.700187,-0.053115,-0.220804,-0.217723,1.0,0.0,1.110607,-0.733359
1,3.177013,-1.369874,0.333637,-3.358735,-1.860428,1.660901,-0.249163,-0.376705,1.298027,-0.219630,...,-0.309108,-0.195153,-0.700187,-0.053115,-0.195153,-0.192087,1.0,0.0,1.110607,-0.733359
2,-0.171393,-0.618852,1.945527,-0.393830,-0.827084,-1.038936,2.970973,-1.244290,0.005170,-3.059465,...,3.276702,-0.175576,3.017263,-0.053115,-0.175576,-0.172523,1.0,0.0,-0.729302,1.284677
3,3.177013,-1.369874,0.333637,-3.358735,-1.860428,1.660901,-0.249163,-0.376705,1.298027,-0.219630,...,-0.309108,-0.230930,-0.700187,-0.286207,-0.230930,-0.227843,1.0,0.0,1.368442,-0.789954
4,3.177013,-1.369874,0.333637,-3.358735,-1.860428,1.660901,-0.249163,-0.376705,1.298027,-0.219630,...,-0.309108,-0.186377,-0.700187,-0.286207,-0.186377,-0.183317,1.0,0.0,1.368442,-0.789954
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42855,-0.078382,-1.754117,0.408609,-1.082111,-0.683564,1.633068,-0.249163,-0.376705,1.298027,-0.219630,...,-0.452540,-0.230930,-0.700187,-0.169661,-0.230930,-0.227843,1.0,0.0,0.013575,0.931525
42856,-0.078382,-1.754117,0.408609,-1.082111,-0.683564,1.633068,0.734767,-2.253110,0.892130,-0.726744,...,-0.452540,-0.230930,-0.700187,-0.053115,-0.230930,-0.227843,1.0,0.0,0.013575,0.931525
42857,-0.078382,-1.754117,0.408609,-1.082111,-0.683564,1.633068,-0.249163,-0.376705,1.298027,-0.219630,...,-0.452540,-0.230930,-0.700187,-0.169661,-0.230930,-0.227843,1.0,0.0,0.013575,0.931525
42858,0.851731,-2.243155,1.439469,-0.923277,-0.583100,-1.400770,-0.159715,-1.688171,0.065303,-0.878878,...,-0.567286,-0.226880,-0.700187,-0.053115,-0.226880,-0.227843,1.0,0.0,1.011355,-0.055263


In [30]:
from sklearn.model_selection import train_test_split

x_normal = df_normal.values
x_attack = df_attack.values

x_normal_train, x_normal_test = train_test_split(
    x_normal, test_size=0.25, random_state=42)

In [31]:
#training on the normal data
#autoencoder 
model = Sequential()
model.add(Dense(30, input_dim=x_normal.shape[1], activation='relu'))
model.add(Dense(3, activation='relu')) # size to compress to
model.add(Dense(30, activation='relu'))
model.add(Dense(x_normal.shape[1])) # Multiple output neurons
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(x_normal_train,x_normal_train,verbose=1,epochs=100)

Metal device set to: Apple M1 Pro
Epoch 1/100


2022-02-20 23:43:52.015610: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-02-20 23:43:52.015777: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2022-02-20 23:43:52.163878: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


  12/1005 [..............................] - ETA: 4s - loss: 1991.1274  

2022-02-20 23:43:52.312898: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1005/1005 [==============================] - 5s 4ms/step - loss: 185.2366
Epoch 2/100
1005/1005 [==============================] - 4s 4ms/step - loss: 38.6912
Epoch 3/100
1005/1005 [==============================] - 4s 4ms/step - loss: 2.5486
Epoch 4/100
1005/1005 [==============================] - 4s 4ms/step - loss: 1.8462
Epoch 5/100
1005/1005 [==============================] - 4s 4ms/step - loss: 1.6416
Epoch 6/100
1005/1005 [==============================] - 4s 4ms/step - loss: 1.5761
Epoch 7/100
1005/1005 [==============================] - 4s 4ms/step - loss: 1.5329
Epoch 8/100
1005/1005 [==============================] - 4s 4ms/step - loss: 1.5201
Epoch 9/100
1005/1005 [==============================] - 4s 4ms/step - loss: 1.4762
Epoch 10/100
1005/1005 [==============================] - 4s 4ms/step - loss: 1.4577
Epoch 11/100
1005/1005 [==============================] - 4s 4ms/step - loss: 1.3596
Epoch 12/100
1005/1005 [==============================] - 4s 4ms/step - loss: 1.308

1005/1005 [==============================] - 4s 4ms/step - loss: 0.6876
Epoch 98/100
1005/1005 [==============================] - 4s 4ms/step - loss: 0.6880
Epoch 99/100
1005/1005 [==============================] - 4s 4ms/step - loss: 0.6957
Epoch 100/100
1005/1005 [==============================] - 4s 4ms/step - loss: 0.6758


In [32]:
pred = model.predict(x_normal_test)
score1 = np.sqrt(metrics.mean_squared_error(pred,x_normal_test))
pred = model.predict(x_normal)
score2 = np.sqrt(metrics.mean_squared_error(pred,x_normal))
pred = model.predict(x_attack)
score3 = np.sqrt(metrics.mean_squared_error(pred,x_attack))
print(f"Out of Sample Normal Score (RMSE): {score1}")
print(f"Insample Normal Score (RMSE): {score2}")
print(f"Attack Underway Score (RMSE): {score3}")

2022-02-20 23:50:58.196384: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


Out of Sample Normal Score (RMSE): 0.8564566460984125
Insample Normal Score (RMSE): 0.822530150239239
Attack Underway Score (RMSE): 1.9537596109886775
